In [41]:
import pandas as pd
import time
import datetime
import lightgbm as lgb

In [42]:
train = pd.read_csv("training_set_VU_DM.csv")

In [43]:
test = pd.read_csv("test_set_VU_DM.csv")

# Data Processing

## Feature engineering part 1:
- date_time
    - convert to unix
    - converst to timestamp format
    - day of week
    - weekend
    - month
- visitor_hist_starrating
    - no_visitor_starrating (when null)
- visitor_hist_adr_usd
    - no_visitor_price (when null)
- prop_starrating
    - not_available_star (when null)
    - no_star (when 0)
- prop_review_score
    - not_available_review (when null)
    - no_review (when 0)
- prop_log_historical_price
    - not_available_price (when 0)
    
...

<ins>Ideas (not implemented)</ins>
- available_competitors (bool)
- available_competitors_count


In [44]:
cols = train.columns

In [45]:
cols

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [46]:
# convert datetime to correct object
train["datetime"] = pd.to_datetime(train["date_time"])
train = train.drop(["date_time"], axis=1)

In [47]:
test["datetime"] = pd.to_datetime(test["date_time"])
test = test.drop(["date_time"], axis=1)

In [48]:
train["datetime_unix"] = train["datetime"].astype(int) / 10**9

In [49]:
test["datetime_unix"] = test["datetime"].astype(int) / 10**9

In [50]:
train["weekday"] = train["datetime"].apply(lambda x: x.dayofweek)

In [51]:
test["weekday"] = test["datetime"].apply(lambda x: x.dayofweek)

In [52]:
train["weekend"] = (train["weekday"]>4).astype(int) 

In [53]:
test["weekend"] = (test["weekday"]>4).astype(int) 

In [54]:
train["month"] = train["datetime"].apply(lambda x: x.month)

In [55]:
test["month"] = test["datetime"].apply(lambda x: x.month)

In [56]:
train = train.drop(["datetime"], axis=1)

In [57]:
test = test.drop(["datetime"], axis=1)

In [58]:
train["no_visitor_starrating"] = train["visitor_hist_starrating"].isna().astype(int)
train["no_visitor_price"] = train["visitor_hist_adr_usd"].isna().astype(int)
train["prop_not_available_star"] = train["prop_starrating"].isna().astype(int)
train["prop_no_star"] = (train["prop_starrating"]==0).astype(int)
train["prop_not_available_review"] = train["prop_review_score"].isna().astype(int)
train["prop_no_review"] = (train["prop_review_score"]==0).astype(int)
train["prop_not_available_price"] = (train["prop_log_historical_price"]==0).astype(int)

In [59]:
test["no_visitor_starrating"] = test["visitor_hist_starrating"].isna().astype(int)
test["no_visitor_price"] = test["visitor_hist_adr_usd"].isna().astype(int)
test["prop_not_available_star"] = test["prop_starrating"].isna().astype(int)
test["prop_no_star"] = (test["prop_starrating"]==0).astype(int)
test["prop_not_available_review"] = test["prop_review_score"].isna().astype(int)
test["prop_no_review"] = (test["prop_review_score"]==0).astype(int)
test["prop_not_available_price"] = (test["prop_log_historical_price"]==0).astype(int)

## Droping columns: 
srch_query_affinity_score, gross_bookings_usd, orig_destination_distance

- at least temporary:
comp1_rate_percent_diff, comp2_rate_percent_diff, comp3_rate_percent_diff, comp4_rate_percent_diff
comp5_rate_percent_diff, comp6_rate_percent_diff, comp7_rate_percent_diff, comp8_rate_percent_diff


In [60]:
train = train.drop(["srch_query_affinity_score", "gross_bookings_usd"], axis=1) 
test = test.drop(["srch_query_affinity_score"], axis=1) 


In [61]:
train = train.drop(["comp1_rate_percent_diff", "comp2_rate_percent_diff", "comp3_rate_percent_diff", 
                    "comp4_rate_percent_diff", "comp5_rate_percent_diff", "comp6_rate_percent_diff", 
                    "comp7_rate_percent_diff", "comp8_rate_percent_diff"], axis=1) 
test = test.drop(["comp1_rate_percent_diff", "comp2_rate_percent_diff", "comp3_rate_percent_diff", 
                    "comp4_rate_percent_diff", "comp5_rate_percent_diff", "comp6_rate_percent_diff", 
                    "comp7_rate_percent_diff", "comp8_rate_percent_diff"], axis=1) 

In [62]:
# create copies
df_train = train
df_test = test

In [63]:
cols

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [64]:
df_train = df_train.drop(["orig_destination_distance"], axis=1) 
df_test = df_test.drop(["orig_destination_distance"], axis=1) 

## Filling in missing values
- comp_rate -> +1 when null because then it is an advantage against the competitor
- comp_inv -> +1 then null because then it is an advantage against the competitor
- user starrating, visitor_hist_adr_usd -> mean
- prop review, location score 2 -> mean

<ins>Ideas (not implemented)</ins>
- orig_destination_distance by looking at other users
- prop review, star = knn/mean region, price
- user rating, usd -> fill in random normal with mean column

In [65]:
for i in range(1, 9):
    rate = "comp"+str(i)+"_rate"
    inv = "comp"+str(i)+"_inv"
    df_train[rate] = df_train[rate].fillna(1)
    df_train[inv] = df_train[inv].fillna(1)
    df_test[rate] = df_test[rate].fillna(1)
    df_test[inv] = df_test[inv].fillna(1)

In [66]:
df_train["visitor_hist_starrating"].fillna(df_train["visitor_hist_starrating"].mean(), inplace=True)
df_test["visitor_hist_starrating"].fillna(df_test["visitor_hist_starrating"].mean(), inplace=True)
df_train["visitor_hist_adr_usd"].fillna(df_train["visitor_hist_adr_usd"].mean(), inplace=True)
df_test["visitor_hist_adr_usd"].fillna(df_test["visitor_hist_adr_usd"].mean(), inplace=True)
df_train["prop_review_score"].fillna(df_train["prop_review_score"].mean(), inplace=True)
df_test["prop_review_score"].fillna(df_test["prop_review_score"].mean(), inplace=True)
df_train["prop_location_score2"].fillna(df_train["prop_location_score2"].mean(), inplace=True)
df_test["prop_location_score2"].fillna(df_test["prop_location_score2"].mean(), inplace=True)

In [67]:
for i in df_train.columns:
    if df_train[i].isna().sum()>0:
        print(i)
        print(df_train[i].isna().sum()/len(df_train)*100)

## Feature engineering part 2

- prop_location_score1 + prop_location_score2
    - prop_location_score_total
- price_usd - prop_log_historical_price
    - prop_price_diff
- srch_adults_count + srch_children_count
    - srch_count
- srch_count/srch_room_count
    - persons_per_room
- price_usd/srch_count
    - price_per_person
- srch_destination_id != visitor_location_country_id
    - srch_abroad
    

- one-hot-encoding cat_features


- avg numeric features per srch_id


- click_bool, booking_bool
    - target_score (5 booked, 1 clicked)    

In [68]:
df_train["prop_location_score_total"] = df_train["prop_location_score1"] + df_train["prop_location_score2"]
df_test["prop_location_score_total"] = df_test["prop_location_score1"] + df_test["prop_location_score2"]

In [69]:
df_train["prop_price_diff"] = df_train["price_usd"] - df_train["prop_log_historical_price"]
df_test["prop_price_diff"] = df_test["price_usd"] - df_test["prop_log_historical_price"]

In [70]:
df_train["srch_count"] = df_train["srch_adults_count"] + df_train["srch_children_count"]
df_test["srch_count"] = df_test["srch_adults_count"] + df_test["srch_children_count"]

In [71]:
df_train["persons_per_room"] = df_train["srch_count"] / df_train["srch_room_count"]
df_test["persons_per_room"] = df_test["srch_count"] / df_test["srch_room_count"]

In [72]:
df_train["price_per_person"] = df_train["price_usd"] + df_train["srch_count"]
df_test["price_per_person"] = df_test["price_usd"] + df_test["srch_count"]

In [73]:
df_train["srch_abroad"] = (df_train["srch_destination_id"] == df_train["visitor_location_country_id"]).astype(int)
df_test["srch_abroad"] = (df_test["srch_destination_id"] == df_test["visitor_location_country_id"]).astype(int)

In [74]:
cat_features = [ "comp1_rate", "comp1_inv", "comp2_rate", "comp2_inv", "comp3_rate", "comp3_inv", "comp4_rate", "comp4_inv",
                "comp5_rate", "comp5_inv", "comp6_rate", "comp6_inv", "comp7_rate", "comp7_inv", "comp8_rate", "comp8_inv",
                "weekday", "month"]
rest = ["srch_id", "site_id", "visitor_location_country_id", "prop_country_id", "prop_id", "srch_destination_id"]

In [75]:
# one-hot encode cat_features
for i in cat_features:
    dummy = pd.get_dummies(df_test[i], prefix=i+'_')
    df_test = df_test.join(dummy)  
df_test = df_test.drop(cat_features, axis=1)

for i in cat_features:
    dummy = pd.get_dummies(df_train[i], prefix=i+'_')
    df_train = df_train.join(dummy)  
df_train = df_train.drop(cat_features, axis=1)

In [76]:
list_1 = list(df_train.columns)
list_2 = list(df_test.columns)
for i in list(set(list_2) - set(list_1)):
    print(i)
for i in list(set(list_1) - set(list_2)):
    print(i)

click_bool
booking_bool
position


In [ ]:
# avg numeric features per srch_id
numeric_cols = ['prop_starrating','prop_review_score','prop_location_score1','prop_location_score2',
                'prop_log_historical_price','price_usd',]

for i in numeric_cols:
    name = i + '_avg'
    df_train[name] = df_train.groupby(['srch_id'])[i].transform('mean')    
    
for i in numeric_cols:
    name = i + '_avg'
    df_test[name] = df_test.groupby(['srch_id'])[i].transform('mean') 

In [77]:
# df_train["target_score"] = df_train["click_bool"] + 5* df_train["booking_bool"]
# df_train = df_train.drop(["click_bool", "booking_bool"], axis=1)

In [78]:
target = "click_bool"

In [79]:
df_train["target_score"] = df_train[target]
df_train = df_train.drop(["click_bool", "booking_bool", "position"], axis=1)

In [81]:
df_train["target_score"] = -df_train["target_score"] #inverts target to make it easier for submission sorting

In [82]:
df_train.head(3)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,weekday__6,month__1,month__2,month__3,month__4,month__5,month__6,month__11,month__12,target_score
0,1,12,187,3.374334,176.022659,219,893,3,3.5,1,...,0,0,0,0,1,0,0,0,0,0
1,1,12,187,3.374334,176.022659,219,10404,4,4.0,1,...,0,0,0,0,1,0,0,0,0,0
2,1,12,187,3.374334,176.022659,219,21315,3,4.5,1,...,0,0,0,0,1,0,0,0,0,0


In [83]:
len(df_train.columns)

102

In [84]:
df_train.to_csv("cleaned_train.csv", index=False)

In [85]:
df_test.to_csv("cleaned_test.csv", index=False)